In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import contextily as ctx
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm

In [ ]:
## Set up directory referencing
from os.path import basename, dirname, exists
# Package data
usr_dir = os.getcwd()
while basename(usr_dir)!='Users':
    temp = basename(usr_dir)
    usr_dir = dirname(usr_dir)
gwfm_dir = usr_dir+'/'+temp+'/Box/research_cosumnes/GWFlowModel/'
map_dir = gwfm_dir+'Mapping/'
sfr_dir = gwfm_dir+'SFR_data/'
lak_data = gwfm_dir+'LAK_data/'
proj_dir = gwfm_dir+'Levee_setback/'
plt_dir = proj_dir+'figures/'

In [ ]:
grid_sfr = gpd.read_file(sfr_dir+'final_grid_sfr/grid_sfr.shp')


In [ ]:
rivers = gpd.read_file(join(sfr_dir,"Sac_valley_rivers/Sac_valley_rivers.shp"))
rivers = rivers.to_crs('EPSG:32610')

mb_regional = gpd.read_file(join(gwfm_dir,"DIS_data/NewModelDomain/GWModelDomain_52_9deg_UTM10N_WGS84.shp"))
# mb = gpd.read_file(join(gwfm_dir,"DIS_data/NewModelDomain/GWModelDomain_52_9deg_UTM10N_WGS84.shp"))
rivers_clip = gpd.clip(rivers, mb_regional)

soam = gpd.read_file(join(map_dir,"so_am_subbasin/so_am_subbasin.shp"))
soam = soam.to_crs('EPSG:32610')
cos = gpd.read_file(join(map_dir,"cos_subbasin/cos_subbasin.shp"))
cos = cos.to_crs('EPSG:32610')

ca = gpd.read_file(join(map_dir,"ca_state_boundary/CA_State_TIGER2016.shp"))
ca = ca.to_crs('EPSG:32610')

In [ ]:
from shapely.geometry import Polygon
def gdf_bnds(df, ax, buf=0):
    lim = df.geometry.unary_union.buffer(buf).bounds
    bnds = Polygon([(lim[0],lim[1]), (lim[2],lim[1]), (lim[2],lim[3]),(lim[0],lim[3])])
    bnds = gpd.GeoDataFrame([0], geometry = [bnds], crs = df.crs)
    ax.set_xlim(lim[0], lim[2])
    ax.set_ylim(lim[1], lim[3])
    return(bnds)
def plt_cln(ax):
    ax.ticklabel_format(style='plain')
    plt.ylabel('Northing (m)')
    plt.xlabel('Easting (m)')
    
    plt.locator_params(axis='x', nbins=5)
    plt.locator_params(axis='y', nbins=5)
    plt.yticks(rotation=90, verticalalignment = "center")


In [ ]:
fig,ax=plt.subplots(figsize=(8,6))
# lak_grid_clip.plot(ax=ax, color='None',edgecolor='blue')
# filter for cells that have more than half left after cropping
lak_grid = lak_grid_clip[lak_grid_clip.geometry.area> 200*200*0.5]
lak_grid.plot(ax=ax, color='None',edgecolor='blue')
ctx.add_basemap(ax=ax,crs='epsg:26910', source=ctx.providers.Esri.WorldImagery,attribution=False)
gdf_bnds(lak_grid,ax)
grid_sfr.plot(ax=ax)

ax.ticklabel_format(style='plain')
plt.xlabel('Easting (m)')
plt.ylabel('Northing (m)')
# grid_sfr.apply(lambda x: ax.annotate(text=x['iseg'], xy=x.geometry.centroid.coords[0], ha='center'), axis=1);

# plt.savefig(plt_dir+'local_floodplain_map.png',dpi=600, bbox_inches='tight')

In [ ]:
setbacks = gpd.read_file(proj_dir+'local_levee_setback_rectangles/local_levee_setback_rectangles.shp')
# setbacks.columns

In [ ]:
fig,ax = plt.subplots(figsize=(8,8))
mb.plot(ax=ax,color="none",edgecolor='black')

gdf_bnds(mb,ax)

rivers.loc[rivers.GNIS_Name.isin(['Mokelumne River','South Mokelumne River'])].plot(ax=ax)
cr = rivers.loc[rivers.GNIS_Name=='Cosumnes River']
cr.plot( ax=ax,label='Cosumnes River')

cr_buf = gpd.GeoDataFrame(pd.DataFrame([0]), geometry = [cr.buffer(3000).unary_union], crs = cr.crs)
cr_buf = gpd.overlay(cr_buf,mb)
cr_buf.plot(color='none',ax=ax)

# rivers.plot( ax=ax,label='Deer Creek',color='navy')
ax.annotate(text='Cosumnes\n River', xy=list(cr.geometry.iloc[10].centroid.coords)[0], 
            xytext = (6,6), textcoords = 'offset pixels',
            bbox=dict(boxstyle="square,pad=0.3", fc="lightgrey", ec="black", lw=2))

# lak_grid_clip.plot(ax=ax,label='Setback Floodplain',color='navy')
# ax.annotate(text='Setback\n Floodplain', xy=list(lak_grid_clip.geometry.unary_union.centroid.coords)[0], 
#             xytext = (6,6), textcoords = 'offset pixels')
#             arrowprops=dict(facecolor='black', headlength = 0.2))

# setbacks.plot(color='None',edgecolor='black',ax=ax)
# setbacks.apply(lambda x: ax.annotate(text=x['location'], xy=x.geometry.centroid.coords[0], ha='center',
#                                     xytext = (-6,-6), textcoords = 'offset pixels'),axis=1);

ctx.add_basemap(ax, source = ctx.providers.Esri.WorldImagery, crs='epsg:26910', alpha = 0.8, attribution=False)
# ctx.add_basemap(ax, source = ctx.providers.Esri.WorldStreetMap, crs='epsg:26910', attribution=False)

x, y, arrow_length = 0.7, 0.15, 0.1
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=20,
            xycoords=ax.transAxes)

ax.ticklabel_format(style='plain')


fontprops = fm.FontProperties(size=18)
scalebar = AnchoredSizeBar(ax.transData,
                           10000, '10 km', 'lower right', 
                           pad=0.1,
                           sep=2,
                           color='black',
                           frameon=False,
                           size_vertical=1,
                           fontproperties=fontprops)
ax.add_artist(scalebar)
plt_cln(ax=ax)
# ax.legend(loc='upper left')
# plt.xlabel('Easting (m)')
# plt.ylabel('Northing (m)')

# first one is CA map
axins = inset_axes(ax, width="80%", height="80%", bbox_to_anchor=(0, .5, .5, .5),
                  bbox_transform=ax.transAxes, loc=2)
# second one is smaller inset
axins2 = inset_axes(ax, width="40%", height="40%", bbox_to_anchor=(0.18, .5, .5, .5),
                  bbox_transform=ax.transAxes, loc=2)
axins.tick_params(labelleft=False, labelbottom=False, left = False, bottom = False)
axins2.tick_params(labelleft=False, labelbottom=False, left = False, bottom = False)

ca.plot(ax = axins,alpha = 0.2)
soam.plot(ax = axins, edgecolor = 'black')
cos.plot(ax = axins, edgecolor = 'black')

cos.plot(ax = axins2, edgecolor = 'black')
soam.plot(ax = axins2, edgecolor = 'black')
mb.plot(ax = axins2, edgecolor = 'black', color = 'none')

# plt_cln(ax=ax)

# plt.savefig(plt_dir+'regional_domain_map.png', dpi = 600, bbox_inches='tight')
# plt.savefig(plt_dir+'setback_blocks_map.png', dpi = 600, bbox_inches='tight')

# ax.ticklabel_format(style='plain')
